# Graph Convolution Network

*Graph Convolution Network (GCN)* architecture is the blueprint of what a GNN looks like.

(Ref. Kipf and Welling in 2017), it is based on the idea of creating an efficient variant of Convolution Neural Networks (CNNs) applied to graphs.

More accurately, it is an approximation of a graph convolution operation in graph signal processing.

More generally, it is the architecture of choice to create a solid baseline when dealing with graph data.

---

"We use the degree matrix to Normalize how to calculate embeddings for a node."

In [3]:
import numpy as np

D = np.array([[3, 0, 0, 0], [0, 1, 0, 0], [0, 0, 2, 0], [0, 0, 0, 2]]) # D matrix -> degree matrix counts the number of neighbours for each node.

D # D gives us the degree of each node, deg(i). Therefore, the inverse of this Matrix D^-1 directly gives us the normalization coefficients 1/deg(i)

array([[3, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 2, 0],
       [0, 0, 0, 2]])

In [6]:
D_inv = np.linalg.inv(D)

D_inv

# We should add self loops to the degree matrix : D` = D + I

# The final matrix we are actually interested in D`^-1 = (D+I)^-1

array([[0.33333333, 0.        , 0.        , 0.        ],
       [0.        , 1.        , 0.        , 0.        ],
       [0.        , 0.        , 0.5       , 0.        ],
       [0.        , 0.        , 0.        , 0.5       ]])

In [10]:
D_tilda_inv = np.linalg.inv(D + np.identity(4))

D_tilda_inv

array([[0.25      , 0.        , 0.        , 0.        ],
       [0.        , 0.5       , 0.        , 0.        ],
       [0.        , 0.        , 0.33333333, 0.        ],
       [0.        , 0.        , 0.        , 0.33333333]])

- $ \tilde{D^{-1}} \tilde{A} X W^{T} $ will normalize every row of features

- $  \tilde{A} \tilde{D^{-1}} X W^{T} $ will normalize every column of features

In [14]:
A = np.array([[1, 1, 1, 1], [1, 1, 0, 0], [1, 0, 1, 1], [1, 0, 1, 1]])

print(np.linalg.inv(D + np.identity(4)) @ A)
print("\n")
print(A @ np.linalg.inv(D + np.identity(4)))

[[0.25       0.25       0.25       0.25      ]
 [0.5        0.5        0.         0.        ]
 [0.33333333 0.         0.33333333 0.33333333]
 [0.33333333 0.         0.33333333 0.33333333]]


[[0.25       0.5        0.33333333 0.33333333]
 [0.25       0.5        0.         0.        ]
 [0.25       0.         0.33333333 0.33333333]
 [0.25       0.         0.33333333 0.33333333]]


The first option looks more appealing because it nicely normalizes neighboring (row) node features and not the single (column) feature

However, Kipf and Welling noticed that features from nodes with a lot of neighbors spread very easily, unlike features from more isolated nodes. In the original GCN paper, the authors proposed a hybrid normalization to counterbalance this effect. In practice, they assign higher weights to nodes with few neighbors using the following formula:

$$ H = \tilde{D}^{-1/2} \tilde{A}^{T} \tilde{D}^{-1/2} X W^{T}$$

In terms of individual embeddings, this operation can be written as follows:

$ h_{i} = \sum_{j \in N_{i}} \frac{1}{\sqrt[]{deg(i)}} \frac{1}{\sqrt[]{deg(j)}} x_{j} W^{T} $

Those are the original formulas to implement a graph convolutional layer. As with our Vanilla GNN layer, we can stack these layers to create a GCN.

## Implementing GCN